In [1]:
# Import library-library
import os
import glob

# Data Preparation and Preprocessing
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
NLTK_StopWords = stopwords.words('indonesian')
NLTK_StopWords = set(NLTK_StopWords)

# Word Embedding
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse

from sklearn.metrics.pairwise import cosine_similarity

# Input and Expansion Query
from textblob import TextBlob
from nltk.tokenize import wordpunct_tokenize
#from googletrans import Translator

#Split to train and test
from sklearn.model_selection import train_test_split

In [2]:
excluded_words = ["tempat", "waktu"]
NLTK_StopWords.update(["detik", "detikjatim", "detikjateng", "detikjabar", "detiksulsel", "detiksumbar", "detikbali", "detikpapua", "detiksulteng", "detikmaluku", "detjatim", "detikcom"])

In [3]:
def preprocessing(berita):
    s = berita.lower()
    s = s.replace('\n', ' ')
    s = s.replace('\r', ' ')
    s = s.replace(' o ', ' ')
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    s = re.sub(r'[0-9]', ' ', s)
    tokens = [token for token in s.split(" ") if token != ""]
    T = [t for t in tokens if ((t in excluded_words) or (t not in NLTK_StopWords))]
    return T

In [4]:
df =pd.read_csv("scrapped_news.csv")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2201 entries, 0 to 2200
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        2201 non-null   object
 1   date         2201 non-null   object
 2   description  2201 non-null   object
 3   source       2201 non-null   object
dtypes: object(4)
memory usage: 68.9+ KB


,title,date,description,source
0,IRT di Badung Bali Curi Perhiasan Senilai Rp 8...,"Senin, 28 Feb 2022 13:32 WIB\n\n",Seorang ibu rumah tangga (IRT) di Kabupaten Ba...,www.detik.com
1,Jejak 'Protagonis' Doni Salmanan Sebelum Terje...,"Jumat, 04 Mar 2022 09:31 WIB\n\n",Soreang itu sempat menjadi bulan-bulanan warga...,www.detik.com
2,Kisah Kejinya Si Pembunuh Berantai 'Twitter'\n\n,"Minggu, 06 Mar 2022 07:09 WIB\n\n",Media sosial bisa menjadi alat yang berbahaya ...,www.detik.com
3,"Wanita Diperkosa 2 Pria Gemparkan Singapura, P...","Minggu, 13 Mar 2022 12:00 WIB\n\n",Seorang wanita berusia 32 tahun menjadi korban...,www.detik.com
4,"Bawa Kabur Mobil Mewah Pelanggan, Karyawan Car...","Jumat, 18 Mar 2022 16:04 WIB\n\n","Karyawan\n\n Dilansir dari\n\n ""Pelaku merupak...",www.detik.com


In [5]:
column = ['Title', 'Date', 'Description', 'Source']
df_total = pd.DataFrame()

df =pd.read_csv("scrapped_news.csv")
isi=df['description']
df = df.dropna(subset=['description'], axis=0)
for x in range(0, df.shape[0]):
    text = preprocessing (df.iloc[x, -2])
    df.iloc[x, -2] = (' '.join(text)) 

In [6]:
df.head()
df_train, df_test=train_test_split(df,test_size=0.1)
df_train.head()

,title,date,description,source
786,Anjasmara Kecelakaan Sepeda Usai Dipepet Orang...,"Selasa, 20 Okt 2020 16:04 WIB\n\n",mengalami pengalaman pahit dipepet orang diken...,www.detik.com
2150,"Perampok Habisi Bos Toko Emas, Pembunuhan Bere...","Rabu, 22 Sep 2021 12:07 WIB\n\n",perampokan pembunuhan bos toko emas kota bandu...,www.detik.com
282,Aksi Heroik Tukang Parkir Sergap Pencuri Modus...,"Senin, 31 Jan 2022 17:49 WIB\n\n",aksi kejahatan jalanan pencurian modus pecah k...,www.detik.com
378,"Kekerasan, Senioritas, dan Kebutuhan Aktualisa...","Kamis, 18 Nov 2021 13:31 WIB\n\n",parade kekerasan tontonan media keseharian fil...,www.detik.com
605,"Sejoli di Jakut Hampir Dibegal, Sempat Terluka...","Selasa, 18 Jan 2022 12:11 WIB\n\n",wanita kekasihnya korban korban putri peristiw...,www.detik.com


In [7]:
desc_text_train = []
desc_text_test = []

for x in range(0, df_train.shape[0]):
  desc_text_train.append(df_train.iloc[x, -2])

for x in range(0, df_test.shape[0]):
  desc_text_test.append(df_test.iloc[x, -2])

In [8]:
print(df_test.info())
print("Panjang Dokumen Test : ", len(desc_text_test))

print ('-'*90)

print(df_train.info())
print("Panjang Dokumen Train : ", len(desc_text_train))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 221 entries, 1935 to 1984
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        221 non-null    object
 1   date         221 non-null    object
 2   description  221 non-null    object
 3   source       221 non-null    object
dtypes: object(4)
memory usage: 8.6+ KB
None
Panjang Dokumen Test :  221
------------------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1980 entries, 786 to 1328
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        1980 non-null   object
 1   date         1980 non-null   object
 2   description  1980 non-null   object
 3   source       1980 non-null   object
dtypes: object(4)
memory usage: 77.3+ KB
None
Panjang Dokumen Train :  1980


In [9]:
# Simpan teks asli dokumen
joblib.dump(desc_text_test, "desc_text_test.pkl")
joblib.dump(desc_text_train, "desc_text_train.pkl")

# Simpan dataframe hasil text processing sebelum TF-IDF Vectorizer
df.to_csv('df_total.csv',index=False )
df_test.to_csv('df_test.csv', index=False)
df_train.to_csv('df_train.csv', index=False)

In [10]:
# Fit transform menggunakan TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_test_matrix = tfidf_vectorizer.fit_transform(desc_text_test)
tfidf_train_matrix = tfidf_vectorizer.fit_transform(desc_text_train)

print('tfidf_test_matrix')
print(tfidf_test_matrix.shape)
print('tfidf_train_matrix')
print(tfidf_train_matrix.shape)


tfidf_test_matrix
(221, 5498)
tfidf_train_matrix
(1980, 17112)


In [11]:
joblib.dump(tfidf_vectorizer, "vectorizer.pkl")
joblib.dump(tfidf_test_matrix, "tfidf_test.pkl")
joblib.dump(tfidf_train_matrix, "tfidf_train.pkl")

['tfidf_train.pkl']

In [12]:
df_total = pd.read_csv('df_total.csv')
df_total = df_total[pd.notnull(df_total['description'])]
print(df_total.info())
print ('-'*90)

document_text_train= joblib.load('desc_text_train.pkl')
document_text_test= joblib.load('desc_text_test.pkl')
print(len(document_text_test))
print(len(document_text_train))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2200 entries, 0 to 2200
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        2200 non-null   object
 1   date         2200 non-null   object
 2   description  2200 non-null   object
 3   source       2200 non-null   object
dtypes: object(4)
memory usage: 85.9+ KB
None
------------------------------------------------------------------------------------------
221
1980


In [25]:
kueri='pencurian'
kueri=preprocessing(kueri)
kueri= [" ".join (kueri)]
print (kueri)

tfidf_matrix =joblib.load( "tfidf_train.pkl" )
tfidf_vectorizer = joblib.load( "vectorizer.pkl" ) 
query_vec= tfidf_vectorizer.transform(kueri)
results=cosine_similarity(tfidf_matrix, query_vec).reshape((-1))
#print (results)
print("\n======================\n")
print("Top 10 most similar documents in corpus:")

j = 1

for i in results.argsort()[-10:][::-1]:
    print(j)
    print("No ID Dokumen  : ", i)
    print("Tanggal        : ", df_total.iloc[i,1])
    print("Isi berita     : ", df_total.iloc[i,2])
    print("(Score: %.4f) " % results[i])
    j += 1

['pencurian']


Top 10 most similar documents in corpus:
1
No ID Dokumen  :  471
Tanggal        :  Minggu, 20 Feb 2022 09:56 WIB


Isi berita     :  motif motif absurd herb mullin membunuh california terkena gempa dikutip murderpedia mullin lahir april mullin normal kecilnya mullin anak cerdas sensitif usia menderita skizofrenia mengamuk membuatnya pembunuh musim panas lulus sma teman mullin dean richardson tewas kecelakaan mobil memicu perilaku anehnya membangun kuil temannya meninggal kamar tidurnya terobsesi reinkarnasi agama bencana alam narkoba kehidupan mullin perilakunya takut keluarga teman temannya didiagnosis penderita skizofrenia paranoid membantu mullin mengeluh mendengar suara suara aneh mengadopsi persona berbeda berganti ganti persona murid yoga petinju amatir
(Score: 0.2597) 
2
No ID Dokumen  :  837
Tanggal        :  Senin, 21 Feb 2022 13:02 WIB


Isi berita     :  bnn provinsi aceh menyelidiki dugaan mengusut indikasi tindak pidana pencucian uang narkoba bisnis jual be